# San Francisco Rental Prices Dashboard
#### In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [3]:
# imports

import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [4]:
# Read the Mapbox API key

load_dotenv()
map_box_api = os.getenv('mapbox')
px.set_mapbox_access_token(map_box_api)

In [5]:
# Import the necessary CSVs to Pandas DataFrames

census_data = pd.read_csv('data_sfo_neighborhoods_census_data.csv', index_col="year")
coordinate_data = pd.read_csv('data_neighborhoods_coordinates.csv').set_index('Neighborhood')

# Panel Visualizations
#### In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard.
#### These functions will convert the plot object to a Panel pane.
#### Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.
#### Return a Panel pane object from each function that can be used to build the dashboard.
#### Note: Remove any .show() lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [6]:
# Define Panel Visualization Functions

def housing_units_per_year():
    avg_units_per_year = census_data.groupby('year').mean()['housing_units']
    fig=px.bar(avg_units_per_year, range_y=(300000,400000), color_discrete_map = {'housing_units' : 'orange'}, title = 'Housing Units in San Francisco, CA from 2010 to 2016'),
    return pn.Pane(fig)

In [7]:
def average_gross_rent():
    avg_gross_rent = census_data.groupby('year').mean()['gross_rent']
    fig=px.line(avg_gross_rent, range_y=(1000,5000), color_discrete_map = {'gross_rent' : 'brown'}, title = 'Average Rent for Housing in San Francisco, CA from 2010 to 2016'),
    return pn.Pane(fig)

In [8]:
def average_sales_price():
    avg_price_per_year = census_data.groupby('year').mean()['sale_price_sqr_foot']
    fig=px.line(avg_price_per_year, range_y=(150,650), color_discrete_map = {'gross_rent' : 'red'}, title = 'Average Sales Price/SQFT Per Year for Housing in San Francisco, CA from 2010 to 2016'),
    return pn.Pane(fig)

In [9]:
def average_price_by_neighborhood():
    avg_price_neighborhood = census_data.groupby(['year' , 'neighborhood']).mean()
    fig = avg_price_neighborhood['sale_price_sqr_foot'].hvplot.line(groupby='neighborhood', title = 'Average Sales Price for Neighborhoods Located in San Francisco, CA from 2010 to 2016')
    return pn.Pane(fig)

In [10]:
def top_most_expensive_neighborhood():
    top_10_neighborhood = census_data.groupby('neighborhood').mean()['sale_price_sqr_foot'].sort_values(ascending=False).head(10)
    fig = px.bar(top_10_neighborhood, range_y=(550,950))
    return pn.Pane(fig)

In [11]:
def most_expensive_neighborhoods_rent_sales():
    avg_price_neighborhood = census_data.groupby(['year', 'neighborhood']).mean()
    fig = avg_price_neighborhood.hvplot.bar(x = 'year', y = ['gross_rent', 'sale_price_sqr_foot'], groupby = 'neighborhood', title = 'Average Rent by Neighborhood in San Francisco, CA')
    return fig

In [12]:
def parallel_coordinates():
    top_10_neighborhood  = census_data.groupby('neighborhood').mean()['sale_price_sqr_foot'].sort_values(ascending = False).head(10)
    top_10_parallel = census_data.groupby('neighborhood').mean().sort_values(by = 'sale_price_sqr_foot', ascending = False).head(10).reset_index()
    fig = px.parallel_coordinates(top_10_parallel, color = 'sale_price_sqr_foot', width = 1500)
    return pn.Pane(fig)

In [13]:
def parallel_categories():
    top_10_neighborhood = census_data.groupby('neighborhood').mean()['sale_price_sqr_foot'].sort_values(ascending = False).head(10)
    top_10_parallel = census_data.groupby('neighborhood').mean().sort_values(by = 'sale_price_sqr_foot', ascending = False).head(10).reset_index()
    fig = px.parallel_categories(top_10_parallel, color = 'sale_price_sqr_foot', width = 1500)
    return pn.Pane(fig)

In [14]:
def neighborhood_map():
    coordinate_data = pd.read_csv('data_neighborhoods_coordinates.csv').set_index('Neighborhood')
    census_data_mean = census_data.rename(str.title, axis = 'columns').groupby('Neighborhood').mean()
    neighborhood_coordinates_merge = pd.merge(coordinate_data, census_data_mean, how = 'inner', on = 'Neighborhood')
    fig = px.scatter_mapbox(neighborhood_coordinates_merge, lat = 'Lat', lon = 'Lon', size = 'Sale_Price_Sqr_Foot', color = 'Gross_Rent', zoom = 11, title = 'Average Sale Price/SQFT and Gross Rent in San Francisco, CA', height = 750, width = 1500)
    return pn.Pane(fig)

In [15]:
def sunburst():
    top_10_neighborhood = census_data.groupby('neighborhood').mean()['sale_price_sqr_foot'].sort_values(ascending = False).head(10)
    top_10_parallel = census_data.groupby('neighborhood').mean().sort_values(by = 'sale_price_sqr_foot', ascending = False).head(10).reset_index()
    most_expensive_neighborhoods = top_10_parallel['neighborhood']
    sunburst_data = census_data[census_data['neighborhood'].isin(most_expensive_neighborhoods)].reset_index()
    fig = px.sunburst(sunburst_data, path = ['year', 'neighborhood'], values = 'sale_price_sqr_foot', color = 'gross_rent', height = 1000, title = 'Cost Analysis of the Most Expensive neighborhoods in San Francisco, CA per Year') 
    return pn.Pane(fig)

In [16]:
# Create a Title for the Dashboard

title = 'SAN FRANCISCO REAL ESTATE DASHBOARD'

In [17]:
# Create a tab layout for the dashboard

average_column = pn.Column(housing_units_per_year(), pn.Row(average_gross_rent(), average_sales_price())),

top_10_column = pn.Column(top_most_expensive_neighborhood(), sunburst()),

coordinate_column = pn.Column(neighborhood_map()),

interactive_column = pn.Column(most_expensive_neighborhoods_rent_sales(), average_price_by_neighborhood()),

parallel_column = pn.Column(parallel_categories(), parallel_coordinates())

In [18]:
# Create the dashboard

sfo_real_estate_dashboard = pn.Tabs(('Average Prices', average_column), ('Top 10', top_10_column), ('Map', coordinate_column), ('Neighborhoods', interactive_column), ('Parallel Comparison', parallel_column))

In [19]:
sfo_real_estate_dashboard.servable()

Tabs
    [0] Str(tuple, name='Average Prices')
    [1] Str(tuple, name='Top 10')
    [2] Str(tuple, name='Map')
    [3] Str(tuple, name='Neighborhoods')
    [4] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)

## Debugging
#### Note: Some of the Plotly express plots may not render in the notebook through the panel functions.
#### However, you can test each plot by uncommenting the following code

In [66]:
# housing_units_per_year()

In [67]:
# average_gross_rent()

In [68]:
# average_sales_price()

In [69]:
# average_price_by_neighborhood()

In [70]:
# top_most_expensive_neighboorhoods()

In [71]:
# most_expensive_neighborhoods_rent_sales()

In [72]:
# neighborhood_map().show()

In [73]:
# parrallel_categories()

In [74]:
# parrallel_coordinates()

In [75]:
# sunburst()